Project 1 - Terrence Cummings

This notebook uses successive API calls to read in Minneapolis tax assessed home values for 2020. 


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import time
from scipy.stats import linregress
import json
from pprint import pprint
from datetime import datetime


In [3]:
#Initialize lists to hold key home sales data
formatted_address = []
community = []
neighborhood = []
ward = []
zoning =[]
landuse = []
parcel_area_sqft = []
property_type = []
totalvalue = []
multiple_uses = []
main_pt = []
buildinguse = []
yearbuilt = []
belowgroundarea = []
abovegroundarea = []
num_stories = []
garage_present = []
primaryheating = []
constructiontype = []
exteriortype = []
roof = []
total_units = []
fireplaces = []
bathrooms = []
bedrooms = []
assessment_year = []
objectid = []


#Initialize API record offset because of 2000 record limit per API call
resultOffset_num = 0

#Base URL for the Open Minnesota database of home sales in Minneapolis
base_url = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Assessors_Parcel_Data_2020/FeatureServer/0/query?where=1%3D1&outFields=FORMATTED_ADDRESS,NEIGHBORHOOD,COMMUNITY,WARD,ZONING,LANDUSE,PARCEL_AREA_SQFT,PROPERTY_TYPE,TOTALVALUE,MULTIPLE_USES,MAIN_PT,BUILDINGUSE,YEARBUILT,BELOWGROUNDAREA,ABOVEGROUNDAREA,NUM_STORIES,GARAGE_PRESENT,PRIMARYHEATING,CONSTRUCTIONTYPE,EXTERIORTYPE,ROOF,TOTAL_UNITS,FIREPLACES,BATHROOMS,BEDROOMS,ASSESSMENT_YEAR,OBJECTID&outSR=4326&f=json'

#Make successive API call to grab 2000 records each time. Total records is 130,719
while resultOffset_num<=150000:

#Increment the record offset to get the next batch of records
    resultOffset = str(resultOffset_num)
    resultRecordCount = str(2000)
    resultOffset_num = resultOffset_num+2000

#Dyamically adjust the offset parameter of the API call and create the next target URL
    pagination_url = f'&resultOffset={resultOffset}&resultRecordCount={resultRecordCount}'
    target_url = f'{base_url}{pagination_url}'

#Make the API call and store data
    home_tax_data = requests.get(target_url).json()


#Set the number of home sales in the data for looping in filling the lists of key data
    num_tax_houses=len(home_tax_data['features'])

#Fill lists of key data
    for house in range(0, num_tax_houses):
        formatted_address.append(home_tax_data['features'][house]['attributes']['FORMATTED_ADDRESS'])
        community.append(home_tax_data['features'][house]['attributes']['COMMUNITY'])
        neighborhood.append(home_tax_data['features'][house]['attributes']['NEIGHBORHOOD'])
        ward.append(home_tax_data['features'][house]['attributes']['WARD'])
        zoning.append(home_tax_data['features'][house]['attributes']['ZONING'])
        landuse.append(home_tax_data['features'][house]['attributes']['LANDUSE'])
        parcel_area_sqft.append(home_tax_data['features'][house]['attributes']['PARCEL_AREA_SQFT'])
        property_type.append(home_tax_data['features'][house]['attributes']['PROPERTY_TYPE'])
        totalvalue.append(home_tax_data['features'][house]['attributes']['TOTALVALUE'])
        multiple_uses.append(home_tax_data['features'][house]['attributes']['MULTIPLE_USES'])
        main_pt.append(home_tax_data['features'][house]['attributes']['MAIN_PT'])
        buildinguse.append(home_tax_data['features'][house]['attributes']['BUILDINGUSE'])
        yearbuilt.append(home_tax_data['features'][house]['attributes']['YEARBUILT'])
        belowgroundarea.append(home_tax_data['features'][house]['attributes']['BELOWGROUNDAREA'])
        abovegroundarea.append(home_tax_data['features'][house]['attributes']['ABOVEGROUNDAREA'])
        num_stories.append(home_tax_data['features'][house]['attributes']['NUM_STORIES'])
        garage_present.append(home_tax_data['features'][house]['attributes']['GARAGE_PRESENT'])
        primaryheating.append(home_tax_data['features'][house]['attributes']['PRIMARYHEATING'])
        constructiontype.append(home_tax_data['features'][house]['attributes']['CONSTRUCTIONTYPE'])
        exteriortype.append(home_tax_data['features'][house]['attributes']['EXTERIORTYPE'])
        roof.append(home_tax_data['features'][house]['attributes']['ROOF'])
        total_units.append(home_tax_data['features'][house]['attributes']['TOTAL_UNITS'])
        fireplaces.append(home_tax_data['features'][house]['attributes']['FIREPLACES'])
        bathrooms.append(home_tax_data['features'][house]['attributes']['BATHROOMS'])
        bedrooms.append(home_tax_data['features'][house]['attributes']['BEDROOMS'])
        assessment_year.append(home_tax_data['features'][house]['attributes']['ASSESSMENT_YEAR'])
        objectid.append(home_tax_data['features'][house]['attributes']['OBJECTID'])


In [14]:
#Create df of all MSP assessed home value data
msp_home_tax_df = pd.DataFrame(zip(objectid, assessment_year, formatted_address, community, neighborhood, ward, zoning, landuse, parcel_area_sqft, property_type, totalvalue, multiple_uses, main_pt, buildinguse, yearbuilt, belowgroundarea, abovegroundarea, num_stories, garage_present, primaryheating, constructiontype, exteriortype, roof, total_units, fireplaces, bathrooms, bedrooms), columns = ['objectid', 'assessment_year', 'formatted_address', 'community', 'neighborhood', 'ward', 'zoning', 'landuse', 'parcel_area_sqft', 'property_type', 'totalvalue', 'multiple_uses', 'main_pt', 'buildinguse', 'yearbuilt', 'belowgroundarea', 'abovegroundarea', 'num_stories', 'garage_present', 'primaryheating', 'constructiontype', 'exteriortype', 'roof', 'total_units', 'fireplaces', 'bathrooms', 'bedrooms'])

msp_home_tax_df.set_index('objectid', inplace=True)

msp_home_tax_df['totalarea'] = msp_home_tax_df['abovegroundarea'] + msp_home_tax_df['belowgroundarea']
msp_home_tax_df

,assessment_year,formatted_address,community,neighborhood,ward,zoning,landuse,parcel_area_sqft,property_type,totalvalue,...,garage_present,primaryheating,constructiontype,exteriortype,roof,total_units,fireplaces,bathrooms,bedrooms,totalarea
objectid,,,,,,,,,,,,,,,,,,,,,
1,2020,4724 FOLWELL DR,LONGFELLOW,HIAWATHA,12,R1,SINGLE-FAMILY DETACHED DW,6719.0,RESIDENTIAL,365000,...,2,FORCED AIR,WOOD FRAME,WOOD,GABLE,1,2,2,2,2436
2,2020,4718 FOLWELL DR,LONGFELLOW,HIAWATHA,12,R1,SINGLE-FAMILY DETACHED DW,7280.0,RESIDENTIAL,378000,...,2,FORCED AIR,WOOD FRAME,WOOD,GABLE,1,1,2,3,2676
3,2020,4712 FOLWELL DR,LONGFELLOW,HIAWATHA,12,R1,SINGLE-FAMILY DETACHED DW,7638.0,RESIDENTIAL,390000,...,2,FORCED AIR,WOOD FRAME,WOOD,GABLE,1,2,2,3,2126
4,2020,4706 FOLWELL DR,LONGFELLOW,HIAWATHA,12,R1,SINGLE-FAMILY DETACHED DW,7695.0,RESIDENTIAL,550500,...,2,FORCED AIR,WOOD FRAME,WOOD,HIP,1,2,3,4,3628
5,2020,4700 FOLWELL DR,LONGFELLOW,HIAWATHA,12,R1,SINGLE-FAMILY DETACHED DW,7467.0,RESIDENTIAL,390000,...,2,FORCED AIR,WOOD FRAME,METAL/VINYL,GABLE,1,2,2,2,2268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130715,2020,3805 46TH AVE S,LONGFELLOW,HOWE,12,R1A,SINGLE-FAMILY DETACHED DW,5208.0,RESIDENTIAL,244500,...,2,FORCED AIR,WOOD FRAME,STUCCO,GABLE,1,0,1,2,1404
130716,2020,3801 46TH AVE S,LONGFELLOW,HOWE,12,R1A,SINGLE-FAMILY DETACHED DW,5208.0,RESIDENTIAL,264000,...,2,FORCED AIR,WOOD FRAME,STUCCO,GABLE,1,0,1,2,1456
130717,2020,3900 RIVER PKWY W,LONGFELLOW,HOWE,12,R1,"INSTITUTIONS:SCHL,COLLEGE",891400.0,COMMERCIAL,0,...,0,STEAM,CONCRETE,BRICK,FLAT,0,0,0,0,90774


In [42]:

#Clean up df by eliminating very low sale price
clean_msp_home_tax_df = msp_home_tax_df.loc[msp_home_tax_df['totalvalue']>10000]

#Clean up df by eliminating very high sale price
clean_msp_home_tax_df = clean_msp_home_tax_df.loc[clean_msp_home_tax_df['totalvalue']<5000000]

#Clean up by eliminating non-residential and odd property types
landuse_allow = ['SINGLE-FAMILY DETACHED DW', 'SINGLE-FAMILY ATTACHED DW']
clean_msp_home_tax_df = clean_msp_home_tax_df[clean_msp_home_tax_df['landuse'].isin(landuse_allow)]

#drop all nan rows
clean_msp_home_tax_df = clean_msp_home_tax_df.dropna()

#drop unbelievable year built
clean_msp_home_tax_df = clean_msp_home_tax_df.loc[clean_msp_home_tax_df['yearbuilt']>1880]

#drop unbelievable totalarea
clean_msp_home_tax_df = clean_msp_home_tax_df.loc[clean_msp_home_tax_df['totalarea']<9000]

#drop zero area
clean_msp_home_tax_df = clean_msp_home_tax_df.loc[clean_msp_home_tax_df['abovegroundarea']>100]

#Add Minneapolis, MN to address
clean_msp_home_tax_df['formatted_address'] = clean_msp_home_tax_df['formatted_address']+', Minneapolis, MN'

In [43]:
#OUTPUT FINAL CLEAN DATA OF 2020 MSP ASSESSED HOME VALUES
clean_msp_home_tax_df.to_csv('/output_data/clean_msp_tax_sales.csv')

In [44]:
clean_msp_home_tax_df.describe()

,assessment_year,parcel_area_sqft,totalvalue,yearbuilt,belowgroundarea,abovegroundarea,garage_present,fireplaces,bathrooms,bedrooms,totalarea
count,93075.0,93075.000000,9.307500e+04,93075.000000,93075.000000,93075.000000,93075.000000,93075.000000,93075.000000,93075.000000,93075.000000
mean,2020.0,18232.520245,3.197124e+05,1939.102380,712.506076,1322.072533,1.360344,0.460650,1.745571,2.756766,2034.578609
std,0.0,43366.004791,2.262259e+05,30.838232,413.264248,556.271330,0.802200,0.679788,0.855468,1.053067,837.772751
min,2020.0,447.000000,2.400000e+04,1883.000000,0.000000,153.000000,0.000000,0.000000,0.000000,0.000000,156.000000
25%,2020.0,5107.000000,1.955000e+05,1916.000000,572.000000,978.000000,1.000000,0.000000,1.000000,2.000000,1562.000000
50%,2020.0,5680.000000,2.650000e+05,1927.000000,794.000000,1212.000000,2.000000,0.000000,2.000000,3.000000,1996.000000
75%,2020.0,7870.500000,3.675000e+05,1954.000000,967.000000,1536.000000,2.000000,1.000000,2.000000,3.000000,2420.000000
max,2020.0,504275.000000,4.995000e+06,2020.000000,4191.000000,7271.000000,8.000000,9.000000,9.000000,15.000000,8991.000000


In [45]:
#A function which takes in: 1) x data, 2) y data, 3) name for x axis, 4) name for y axis, 5) indicate or whether 
#to draw linear regression or not: TRUE = include linear regression plot, FALSE = do not include linear regression.abs

def get_line(x,y,x_name,y_name, incl_regress):
    
    #Initialize the two plats: ax1=scatter, ax2=linear regression line    
    ax1=[]
    ax2=[]
    
    #count the number of cities in the plot to include in the title
    plot_count = x.count()
    
    #Do scatter plot, titel, and labels
    ax1=plt.scatter(x,y)
    ax1=plt.title(f'{plot_count} plots of {y_name} vs. {x_name}')
    ax1=plt.xlabel(f'{x_name}')
    ax1=plt.ylabel(f'{y_name}')
    
    #Include linear regression plot if flag indicates 
    if incl_regress:
        #Perform linear regression and r2 and plot
        slope, intercept, rvalue, pvalue, stderr = st.linregress(x,y)
        r_squared = rvalue**2
        ax2=plt.plot(x, x*slope+intercept, color='red', label= 'Y= '+str(round(slope,2))+'*X + '+str(round(intercept,2))+' R2='+str(round(r_squared,2)))
        #Use legend to show the linear regression equation and R2 value
        ax2=plt.legend()
        
    
    return ax1, ax2

In [36]:
#CREATE A TABLE OF ASSESSED HOME VALUE PER SQFT BY MSP NEIGHBORHOOD (87 NEIGHBORHOODS, 11 COMMUNITIES)
nbhd = pd.DataFrame(pd.read_csv('/output_data/clean_msp_tax_sales.csv'))
nbhd['val_tot_sqft'] = nbhd['totalvalue']/nbhd['totalarea']
nbhd_grp = nbhd.groupby('neighborhood')

nbhd_grp_df = pd.DataFrame(nbhd_grp['totalvalue'].count())
nbhd_grp_df.rename(columns={'totalvalue':'Number of Homes'}, inplace=True)
nbhd_grp_df['val_tot_sqft'] = nbhd_grp['val_tot_sqft'].mean()
nbhd_grp_df

nbhd_grp_df.to_csv('/output_data/nbhd_val_tot_sqft.csv')